# Comparing between different Sentimental Analaysis Models

Goal: To determine the best model on hand

Description:  I have a csv of headliners, and fed it to GrokAI to generate a list of sentimental Scores. This will be used as the benchmark in the comparison.

Steps outlined:
1. Setup the file "with_sentiment_100.csv" for comparison and briefly screen through the list for outliers
2. Run through the different models and run the data through them (We are interested to know if its positive or negative)
3. Compare with true values

Models:
- vader
- textblob
- flair
- roberta
- distilbert
- bertweet
- finbert
- deberta
- qwen llm

## 1. Setup testing file

In [1]:
import pandas as pd
df = pd.read_csv("data/headlines_with_sentiment.csv")
df.head()

,Index,Headline,Sentiment
0,0,Johnson is asking Santa for a Christmas recovery,POS
1,1,‘I now fear the worst’: four grim tales of wor...,NEG
2,2,Five key areas Sunak must tackle to serve up e...,NEU
3,3,Covid-19 leaves firms ‘fatally ill-prepared’ f...,NEG
4,4,The Week in Patriarchy Bacardi's 'lady vodka':...,NEG


In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [2]:
headline = list(df['Headline'])[:10]
headline

['Johnson is asking Santa for a Christmas recovery',
 '‘I now fear the worst’: four grim tales of working life upended by Covid-19',
 'Five key areas Sunak must tackle to serve up economic recovery',
 'Covid-19 leaves firms ‘fatally ill-prepared’ for no-deal Brexit',
 "The Week in Patriarchy Bacardi's 'lady vodka': the latest in a long line of depressing gendered products",
 'English councils call for smoking ban outside pubs and cafes',
 'Can Tesla justify a $300bn valuation?',
 "Empty city centres: 'I’m not sure it will ever be the same again'",
 "Democratising finance for all? An investment app for amateurs and a student trader's death",
 'Homebuyer loses £300,000 to fraudsters – but gets it back after we step in']

## 2. Run the list through different models

In [4]:
# set up

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    if not isinstance(text, str) or text is None:
        return ""
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    cleaned_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
    test_sentence = " ".join(cleaned_tokens)
    return test_sentence

processed_headine = list(map(preprocess, headline))

[nltk_data] Downloading package wordnet to /Users/jaytai/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaytai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/jaytai/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
# 1. Prebuilt Vader sentiment package (NaiveBayes model) - Done

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
result_vader = []
threshold_upper = .05
threshold_lower = -.05
analyzer = SentimentIntensityAnalyzer()
for sentence in processed_headine:
    score = analyzer.polarity_scores(sentence)
    result_vader.append(score['compound'])
result_vader = ['POS' if s >= threshold_upper else 'NEG' if s <= threshold_lower else 'NEU' for s in result_vader]
print(result_vader[:10])
pd.DataFrame(result_vader).to_csv("data/result_vader.csv")

# to determine the threshold

['NEU', 'NEG', 'NEU', 'NEG', 'NEG', 'NEG', 'NEU', 'POS', 'NEG', 'NEG']


In [ ]:
# 2. Prebuilt Textblob sentiment package - Done

from textblob import TextBlob
result_tb = []
threshold_upper = .05
threshold_lower = -.05
for sentence in processed_headine:
    result_tb.append(TextBlob(sentence).sentiment.polarity)
result_tb = ['POS' if s >= threshold_upper else 'NEG' if s <= threshold_lower else 'NEU' for s in result_tb]
print(result_tb[:10])
pd.DataFrame(result_tb).to_csv("data/result_tb.csv")

# to determine the threshold

In [ ]:
# 3. Prebuilt Flair sentiment package/Model - Done

from flair.data import Sentence
from flair.nn import Classifier
result_flair = []
tagger = Classifier.load('sentiment')

for sentence in processed_headine:
    sentence = Sentence(sentence)
    tagger.predict(sentence)
    value = sentence.labels[0].value
    result_flair.append(value)
result_flair = ['POS' if s == 'POSITIVE' else 'NEG' if s == 'NEGATIVE' else 'NEU' for s in result_flair]
print(result_flair[:10])
pd.DataFrame(result_flair).to_csv("data/result_flair.csv")

In [5]:
# setup for HuggingFace Transformers

from transformers import pipeline, set_seed
set_seed(999)

In [ ]:
# RoBERTa - Done

classifier = pipeline('sentiment-analysis', model='cardiffnlp/twitter-roberta-base-sentiment-latest')

result_roberta = []
for sentence in processed_headine:
    temp = classifier(sentence)
    result_roberta.append(temp[0]['label'])
result_roberta = ['POS' if s == 'positive' else 'NEG' if s == 'negative' else 'NEU' for s in result_roberta]
print(result_roberta[:10])
pd.DataFrame(result_roberta).to_csv("data/result_roberta.csv")

In [ ]:
# distilBERT - Done

classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

result_dis = []
for sentence in processed_headine:
    temp = classifier(sentence)
    result_dis.append(temp[0]['label'])
result_dis = ['POS' if s == 'POSITIVE' else 'NEG' if s == 'NEGATIVE' else 'NEU' for s in result_dis]
print(result_dis[:10])
pd.DataFrame(result_dis).to_csv("data/result_distilbert.csv")

In [ ]:
# Bertweet - Done

classifier = pipeline("text-classification", model="finiteautomata/bertweet-base-sentiment-analysis")

result_bertweet = []
for sentence in processed_headine:
    result_bertweet.append(classifier(sentence)[0]['label'])
print(result_bertweet[:10])
pd.DataFrame(result_bertweet).to_csv("data/result_bertweet.csv")

In [9]:
# finBERT - Done

classifier = pipeline("sentiment-analysis", model="ProsusAI/finbert")

result_finbert = []
for sentence in processed_headine:
    temp = classifier(sentence)
    result_finbert.append(temp[0]['label'])
result_finbert = ['POS' if s == 'positive' else 'NEG' if s == 'negative' else 'NEU' for s in result_finbert]
print(result_finbert[:10])
pd.DataFrame(result_finbert).to_csv("data/result_finbert.csv")

Device set to use mps:0


['NEU', 'NEU', 'POS', 'NEU', 'NEG', 'NEU', 'NEU', 'NEU', 'NEU', 'NEG']


In [ ]:
# LLM QWEN 8gb
# 
# from openai import OpenAI
# 
# client = OpenAI(
#   base_url="https://openrouter.ai/api/v1",
#   api_key="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",
# )
# 
# result_ai = []
# 
# for sentence in processed_headine:
#     completion = client.chat.completions.create(
#       extra_body={}, model="deepseek/deepseek-r1-0528-qwen3-8b:free",
#       messages=[{ "role": "user",
#           "content": f"Only give me a sentimental analysis from -1(negative) to 1(positive) for the following sentence {sentence}"
#     }])
#     result_ai.append(completion.choices[0].message.content)
# result_ai[:10]
# pd.DataFrame(result_ai).to_csv("data/ai.csv")

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

completion = client.chat.completions.create(
      extra_body={}, model="deepseek/deepseek-r1-0528-qwen3-8b:free",
      messages=[{ "role": "user",
          "content": f"Only give me a sentimental analysis value of NEG (negative) or POS (positive) or NEU (neutral) for the following sentence {"Hi Hi"}"
    }])
print(completion.choices[0].message.content)